# CSMODEL
### Project 1 - Group 5
Members:
- Alon-alon, Jason Miguel E.
- Escalona, Jose Miguel A.
- Llorca, Marcelino Francis S.

### 1. Dataset Representation

- About the Dataset<br><br>
    The data was provided by Our World in Data (OWID). The file contains different data values that could help paint a better image of a country’s status for COVID-19. The version used in this project will be the July 15, 2021 release of the dataset, however OWID attempts both daily and weekly update of data whenever possible, thus ensuring that the data they provide is the latest possible.
    <br><br>
- Collection Process and its Implications<br><br>
    {CONTENT}
    <br><br>
- Structure of Dataset of the File<br><br>
    {CONTENT}
    <br><br>
- About the Variables<br><br>
    {CONTENT}

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import ttest_ind

#Code for data preparation
raw_df = pd.read_csv('COVID_7_15.csv')
print("Raw Dataframe Shape:", raw_df.shape)

Raw Dataframe Shape: (102475, 60)


### 2. Data Cleaning

In [2]:
#Code for data cleaning

### 3. Exploratory Data Analysis

EDA Questions<br><br>
1. Do case trends increase/decrease on every listed countries by month?
2. Is there a correlation between the GDP per capita to hospital and ICU patients of a country?
3. Do case numbers correlate negatively with the number of people being vaccinated?

**Numerical Summaries**

In [3]:
#Code for numerical summaries

**Visualization**

In [4]:
#Code for visualizations

### 4. Research Question

1. Is there a significant difference between ASEAN member nations in total and new case numbers?<br><br>
    1. Scope in Dataset: Total cases and/or New cases
    2. Significance: This is in order to know how the Philippines fare against COVID-19 in comparison to our neighboring countries in the ASEAN as well as in the world.

2. Is the government meeting its half-way goal of vaccinating a significant number of people?<br><br>

### 5. Statistical Inference

**Hyptothesis**<br><br>

$H_0=$ 
<br>
$H_A=$ 
<br>

In [5]:
#Code for formulating statistical inference and hypothesis testing

### 6. Insights and Conclusions

### Sources:
- Hannah Ritchie, Esteban Ortiz-Ospina, Diana Beltekian, Edouard Mathieu, Joe Hasell, Bobbie Macdonald, Charlie Giattino, Cameron Appel, Lucas Rodés-Guirao and Max Roser [Online Resource]. (2020). *Coronavirus Pandemic (COVID-19)*. Retrieved from: 'https://ourworldindata.org/coronavirus' 
- Our World in Data [Github Repository]. (2020). COVID-19 Dataset by Our World in Data. Retrieved from: https://github.com/owid/covid-19-data.git



====Baka kailanganin lang====<br>
- Emergency Use Authorization. Food and Drug Administration - Philippines. (2021, July 11). https://www.fda.gov.ph/list-of-fda-issued-emergency-use-authorization/. 